In [1]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle fastai py7zr

In [2]:
from fastkaggle import *

In [3]:
# for working with paths in Python, I recommend using `pathlib.Path`
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not iskaggle and not cred_path.exists():
    creds = ''
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [4]:
comp = 'kkbox-music-recommendation-challenge'

path = setup_comp(comp, install='fastai')

In [5]:
path

Path('kkbox-music-recommendation-challenge')

In [6]:
from fastai.tabular.all import *
from fastai.collab import *

path.ls()

(#6) [Path('kkbox-music-recommendation-challenge/test.csv.7z'),Path('kkbox-music-recommendation-challenge/songs.csv.7z'),Path('kkbox-music-recommendation-challenge/sample_submission.csv.7z'),Path('kkbox-music-recommendation-challenge/train.csv.7z'),Path('kkbox-music-recommendation-challenge/members.csv.7z'),Path('kkbox-music-recommendation-challenge/song_extra_info.csv.7z')]

In [ ]:
for file in path.ls(file_type="7z"):
    !py7zr x {str(file)} {str(path)}

In [12]:
cats=['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-root', 'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season']
conts=['cap-diameter', 'stem-height', 'stem-width']
dep='class'

In [13]:
dls = TabularDataLoaders.from_csv(path/'train.csv', path=path, y_names=dep,
    cat_names=cats,
    cont_names=conts,
    procs=[Categorify, FillMissing, Normalize])

dls.show_batch()

/nix/store/dh9zjc820sfsz49pz3sld5x67wv3xa0c-python3-3.11.9-env/lib/python3.11/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,cap-diameter_na,cap-diameter,stem-height,stem-width,class
0,c,g,n,f,a,#na#,w,#na#,#na#,g,#na#,#na#,f,f,#na#,l,a,False,1.67,5.32,1.619999,e
1,b,d,y,f,a,#na#,y,#na#,t,y,#na#,#na#,f,f,#na#,g,a,False,3.09,4.82,5.970000,p
2,f,l,n,f,x,c,n,#na#,#na#,n,#na#,#na#,f,f,#na#,l,a,False,4.06,5.93,5.060000,p
3,s,d,o,f,d,c,o,#na#,#na#,o,#na#,#na#,f,f,#na#,d,u,False,2.56,2.71,5.450000,p
4,x,d,e,f,a,c,e,#na#,#na#,o,#na#,#na#,f,f,#na#,g,w,False,3.54,4.64,6.390000,e
5,x,t,r,t,d,c,n,#na#,#na#,n,#na#,#na#,f,f,#na#,d,a,False,10.44,6.63,14.710000,p
6,x,d,b,f,d,d,b,#na#,#na#,w,#na#,#na#,f,f,#na#,d,w,False,4.73,4.78,11.590000,e
7,s,#na#,r,t,d,#na#,o,#na#,#na#,w,#na#,#na#,t,f,#na#,d,u,False,7.87,4.53,12.090000,p
8,c,i,y,f,#na#,c,n,#na#,#na#,w,#na#,#na#,f,f,#na#,d,u,False,5.42,6.51,7.870000,p
9,x,t,k,f,a,d,y,#na#,#na#,w,#na#,#na#,f,f,w,d,a,False,15.48,7.18,21.389999,e


In [14]:
from sklearn.metrics import matthews_corrcoef

def corrcoef(inp, targ):
    prediction = inp.argmax(dim=1).unsqueeze(1)
    return matthews_corrcoef(targ, prediction)

learn = tabular_learner(dls, metrics=corrcoef)

In [15]:
learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,corrcoef,time
0,0.057743,0.041832,0.981764,06:18
1,0.037071,0.041958,0.982585,06:05
2,0.040849,0.041316,0.982445,06:08
3,0.041992,0.041478,0.982494,06:10
4,0.047534,0.040896,0.982893,06:05
5,0.044003,0.040513,0.982847,06:56
6,0.043277,0.040331,0.983061,08:23
7,0.043729,0.039833,0.983130,08:16
8,0.038930,0.040778,0.982883,06:44
9,0.033648,0.040702,0.983151,06:07


In [16]:
learn.show_results()

,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,cap-diameter_na,cap-diameter,stem-height,stem-width,class,class_pred
0,49.0,82.0,64.0,9.0,0.0,29.0,44.0,0.0,0.0,56.0,0.0,0.0,6.0,19.0,0.0,26.0,4.0,1.0,-0.672437,0.063541,-1.061750,0.0,0.0
1,72.0,57.0,64.0,9.0,76.0,29.0,47.0,0.0,52.0,47.0,0.0,0.0,19.0,29.0,0.0,26.0,1.0,1.0,-0.009005,-0.655392,-0.656780,0.0,0.0
2,54.0,57.0,64.0,9.0,76.0,0.0,60.0,30.0,40.0,47.0,0.0,0.0,6.0,19.0,0.0,26.0,3.0,1.0,-0.296707,1.975755,-0.581466,1.0,1.0
3,68.0,65.0,64.0,9.0,48.0,29.0,62.0,0.0,45.0,47.0,0.0,0.0,6.0,19.0,0.0,26.0,3.0,1.0,1.910440,-0.121751,2.974368,1.0,1.0
4,49.0,60.0,65.0,9.0,53.0,0.0,62.0,0.0,0.0,44.0,0.0,0.0,6.0,19.0,0.0,30.0,1.0,1.0,-0.264502,0.222892,-0.472815,1.0,1.0
5,72.0,60.0,64.0,9.0,45.0,0.0,47.0,31.0,0.0,47.0,0.0,8.0,6.0,19.0,0.0,31.0,1.0,1.0,0.373166,1.879403,0.654433,1.0,1.0
6,68.0,77.0,67.0,21.0,48.0,29.0,47.0,0.0,0.0,47.0,0.0,0.0,6.0,19.0,0.0,26.0,3.0,1.0,0.774661,-0.103222,0.642087,1.0,1.0
7,54.0,80.0,64.0,9.0,0.0,32.0,44.0,0.0,0.0,56.0,0.0,0.0,6.0,19.0,0.0,26.0,1.0,1.0,-1.033138,-0.466394,-1.071628,0.0,0.0
8,72.0,77.0,64.0,21.0,67.0,0.0,62.0,0.0,0.0,47.0,0.0,0.0,6.0,19.0,0.0,26.0,3.0,1.0,1.199773,1.282763,1.535984,0.0,0.0


In [17]:
test_df = pd.read_csv(path/'test.csv')
stem_height_mode = test_df.mode().iloc[0]['stem-height']
test_df['stem-height'] = test_df['stem-height'].fillna(stem_height_mode)

test_dl = learn.dls.test_dl(test_df)
output = learn.get_preds(dl=test_dl, with_decoded=True)
probs,_,idxs = output

/nix/store/dh9zjc820sfsz49pz3sld5x67wv3xa0c-python3-3.11.9-env/lib/python3.11/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [18]:
vocab = np.array(learn.dls.vocab)
results = pd.Series(vocab[idxs], name="class")
results

0          e
1          p
2          p
3          p
4          e
          ..
2077959    p
2077960    p
2077961    p
2077962    e
2077963    e
Name: class, Length: 2077964, dtype: object

In [19]:
sample_submission = pd.read_csv(path/'sample_submission.csv')
sample_submission

,id,class
0,3116945,e
1,3116946,e
2,3116947,e
3,3116948,e
4,3116949,e
...,...,...
2077959,5194904,e
2077960,5194905,e
2077961,5194906,e
2077962,5194907,e


In [20]:
test_df['class'] = results

sub_df = test_df[['id','class']]
sub_df.to_csv(f'submission.csv', index=False)

In [21]:
!head submission.csv

id,class
3116945,e
3116946,p
3116947,p
3116948,p
3116949,e
3116950,e
3116951,e
3116952,p
3116953,p


In [22]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('submission.csv', 'poisonous-mushrooms-05', comp)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19.8M/19.8M [00:23<00:00, 903kB/s]


In [23]:
torch.save(learn.model.state_dict(), '02.pth')

In [5]:
if not iskaggle:
    push_notebook('alexchalk', 'poisonous-mushrooms-submission-05',
                  title='Poisonous Mushrooms Submission 05',
                  file='02.ipynb',
                  competition=comp, private=True, gpu=False)

Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/alexchalk/poisonous-mushrooms-submission-05
